# パラメータ指定ツール

### **Usage**

```main(file = "自動計測データ.xlsx", target=10)```

自動計測データファルの個体ごとの設定パラメータのうち、初期配置の回転角と上下カット量を GUI で指定するプログラムです。

> **file** 司令ファイル名  
> **target**  チェック欄の数値　この数値がチェック欄に記載されている個体が処理対象となります。


まず、回転角をマウス操作で確定したのち、上下カット量を矩形で指定して確定

ESC 終了書き込み  
ENTER １つの個体処理の確定、つぎの個体へ
R リセット（現在の対象の処理の最初の状態に戻す）
→　１度右回転  
←　１度左回転  
↑　５度左回転  
↓　５度右回転  

[![Tool](https://monosnap.com/image/TsCYn8ZcnHcQVY62HSHViSCcH53UDP.png)](https://www.youtube.com/watch?v=E4WwWzWe_so)


- 上のプログラムは回転が先で、画像を回転させた状態で上下幅指定

- 下のプログラムはフレーミングが先で画像固定


In [15]:
import numpy as np
import cv2
import pandas as pd
import os
from rdlib2 import *
import matplotlib.pyplot as plt
%matplotlib inline
def main1(target=10,file="自動計測データ.xlsx"):
    global df,rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,angle,color2,gx,gy,x0,y0,height,width
    df = pd.read_excel(file)
    
    for radish in range(len(df)):
        
        def initcondition():
            global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect
            rect = (0,0,1,1)
            anglemode = False         # 角度モードオン
            framing = False           # 選択枠設定中
            framed = False       # 枠設定は完了している
            drawing = False
            angling = True
            angled = False
            lx,ly,dx,dy =0,0,0,0
            angle = 0
            
        initcondition()
        idata = df.iloc[radish]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        check = idata['処理対象'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        if check !=target : #  check が 10 でない画像はスルーする
                continue

        path = os.path.join(topdir,subdir,filename)
        print("処理対象画像 {}\n".format(path))
        src= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        org= cv2.imread(os.path.join('一軍',subdir,filename),1)
        org = cv2.resize(org,(int(org.shape[1]/2),int(org.shape[0]/2)))
        
        rdimg = getstandardShape(src, unitSize=UNIT, thres=0.25, setrotation = 0, norotation = True)
        _img,contours,hierarchy = cv2.findContours(rdimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[np.argmax([len(c) for c in contours])] # 輪郭線情報　global 変数　
        
        x0,y0,width,height = cv2.boundingRect(rdcnt) # 外接矩形
        print("x0 {} y0 {} width {} height {}".format(x0,y0,width,height))

        color = cv2.cvtColor(rdimg, cv2.COLOR_GRAY2BGR)
        color2 = color.copy()

        mu = cv2.moments(rdimg, False)
        gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"])
      
        wname = "Angles"

        angle = 0
        while True:
                img = color2.copy()
                rect = (0,0,img.shape[1],img.shape[0])
                cv2.namedWindow(wname)
                cv2.namedWindow(filename)
                cv2.setMouseCallback(wname, onMouse, [wname,img,gx,gy])
                cv2.imshow(filename,org)
                cv2.moveWindow(filename, 0, 500)
                cv2.imshow(wname, color)
                key = cv2.waitKey(0)
                if key==ord('r'):
                    initcondition()
                    color2=color.copy()
                    continue
                elif key==ord('b'):
                    framing == True
                    continue
                elif key==2: # LEFT
                    color2 = imgrotation(1,(gx,gy),color2)
                    angle = angle+1
                elif key==3: # RIGHT
                    color2 = imgrotation(-1,(gx,gy),color2)
                    angle = angle-1
                elif key==0: # UP
                    color2 = imgrotation(5,(gx,gy),color2)
                    angle = angle+5
                elif key==1: # DOWN
                    color2 = imgrotation(-5,(gx,gy),color2)  
                    angle = angle-5
                elif key==13: # Enter で確定　次へ
                    head100 = df.loc[radish,'CCUT0']=int(100*(head-y0)/height) 
                    if head100 <0 :
                        head100 = 3
                    elif head100 >80 :
                        head100 = 80
                    df.loc[radish,'CCUT0']=head100
                    bottom100 = int(100*(bottom-y0)/height) 
                    if bottom100 > 100:
                        bottom100 = 97
                    elif bottom100 < 20:
                        bottom100 = 20
                    df.loc[radish,'TCUT0']= bottom100
                    df.loc[radish,'ROT']=int(angle)
                    df.loc[radish,'処理対象']=1
                    print("angle {0:0.1f} head {1}->{2} bottom {3}->{4}".format(angle,int(100*(head-y0)/height),\
                                                                head100,int(100*(bottom-y0)/height),bottom100))
                    df.to_excel(file, index=True, header=True)
                    break
                elif key==27: # ESC で終了
                    df.to_excel(file, index=True, header=True)
                    break
        if key==27: # ESC で終了
                cv2.destroyAllWindows()
                break
        
    cv2.waitKey(1)

def imgrotation(angle,center,img_src):
        global x0,y0,width,height
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
        size=tuple([img_src.shape[1], img_src.shape[0]])
        mg_rot = cv2.warpAffine(img_src, rotation_matrix, size, flags=cv2.INTER_CUBIC)
        gry = cv2.cvtColor(mg_rot,cv2.COLOR_BGR2GRAY)
        _img,contours,hierarchy = cv2.findContours(gry, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[np.argmax([len(c) for c in contours])] # 輪郭線情報　global 変数　
        x0,y0,width,height = cv2.boundingRect(rdcnt) # 外接矩形
        return mg_rot

def onMouse(event, x, y, flag, params):
    global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,lx,ly,dx,dy,color2,gx,gy,x0,y0,height,width
    wname, img, gx,gy= params
    if angled == False:
        if event == cv2.EVENT_LBUTTONDOWN:
            angling == True
        elif event == cv2.EVENT_LBUTTONUP:
                if angling == True:
                    dx = gx - x
                    dy = gy - y
                    angled = True
                    angling = False
                    print("角度確定",dx,dy)
                    if dx == 0 :
                        angle = angle        
                    else:
                        if dx > 0:
                            angle = angle + 180*np.arctan(dy/dx)/np.pi-90
                        else:
                            angle = angle + 180*np.arctan(dy/dx)/np.pi+90
                    color2 = imgrotation(angle,(gx,gy),color.copy())
                    print("imgrotation")
        elif event == cv2.EVENT_MOUSEMOVE:
                if angling == True : 
                    img = color2.copy()
                    h, w = img.shape[0], img.shape[1]
                    if x!=gx:
                        cv2.line(img, (x-500, y-int(500*(y-gy)/(x-gx))), (gx+500, gy+int(500*(y-gy)/(x-gx))), (255, 0, 0))
                    cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))
                    cv2.circle(img, (gx,gy), 3, (255,0,0), 3)
                    cv2.circle(img, (x,y), 3, (255,0,0), 3)   
    elif framed == False: 
        if event == cv2.EVENT_LBUTTONDOWN:
            framing = True # 矩形描画モードオン
            lx,ly = x,y
            print("矩形描画スタート")
        elif event == cv2.EVENT_MOUSEMOVE: 
            img = color2.copy()
            h, w = rdimg.shape[0], rdimg.shape[1]
            cv2.line(img, (x,0),(x,h-1),(255, 0, 0))
            cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))       
            if framing == True :                      
                # img = color.copy()
                cv2.rectangle(img,(lx,ly),(x,y),(0,0,255),2)
                rect = (min(lx,x),min(ly,y),abs(lx-x),abs(ly-y))  
        elif event == cv2.EVENT_LBUTTONUP:
            if framing == True:
                framing = False
                framed = True
                print("フレーム一時確定")
                cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)
                head = rect[1]
                bottom = rect[1]+rect[3]
                print("angle {0:0.1f} head {1} bottom {2}".format(angle,int(100*(head-y0)/height),int(100*(bottom-y0)/height)))
    cv2.imshow(wname, img)

In [16]:
main1(file="自動計測データAllAuto.xlsx",target=3)
cv2.destroyAllWindows()
cv2.waitKey(1)

処理対象画像 シルエット/17Cylindric/17yukibi2o05_l.jpg

x0 144 y0 65 width 96 height 250


-1

In [17]:
import numpy as np
import cv2
import pandas as pd
import os
from rdlib2 import *

def main2(target=10,file="自動計測データ2.xlsx"):
    global df,rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,angle
    df = pd.read_excel(file)
    
    for radish in range(len(df)):
        
        def initcondition():
            global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect
            rect = (0,0,1,1)
            anglemode = False         # 角度モードオン
            framing = False           # 選択枠設定中
            framed = False       # 枠設定は完了している
            drawing = False
            angling = False
            angled = False
            lx,ly,dx,dy =0,0,0,0
            angle = 0
        
        def imgrotation(angle,center,img_src):
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
            size=tuple([img_src.shape[1], img_src.shape[0]])
            mg_rot = cv2.warpAffine(img_src, rotation_matrix, size, flags=cv2.INTER_CUBIC)
        
        
        initcondition()
        idata = df.iloc[radish]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        check = idata['処理対象'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        
        if check !=target : #  check が 10 でない画像はスルーする
                continue

        path = os.path.join(topdir,subdir,filename)
        print("処理対象画像 {}\n".format(path))
        src= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            
        rdimg = getstandardShape(src, unitSize=UNIT, thres=0.25, setrotation = 0, showResult=False)
        _img,contours,hierarchy = cv2.findContours(rdimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[0] # 輪郭線情報　global 変数　
        
        x0,y0,width,height = cv2.boundingRect(rdcnt) # 外接矩形

        color = cv2.cvtColor(rdimg, cv2.COLOR_GRAY2BGR)

        mu = cv2.moments(rdimg, False)
        gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"])
      
        wname = "Angles"

        while True:
                img = color.copy()
                rect = (0,0,img.shape[1],img.shape[0])
                cv2.namedWindow(wname)
                cv2.setMouseCallback(wname, onMouse, [wname,img,gx,gy])
                cv2.imshow(wname, color)
                key = cv2.waitKey(0)
                if key==ord('r'):
                    initcondition()
                    continue
                elif key==13: # Enter で確定　次へ
                    print(head,bottom,angle,x0,height)
                    df.loc[radish,'CCUT0']=int(100*(head-y0)/height) if int(100*(head-y0)/height) > 0 else 3
                    df.loc[radish,'TCUT0']=int(100*(bottom-y0)/height) if int(100*(bottom-y0)/height) < 100 else 98
                    df.loc[radish,'ROT']=int(angle)
                    df.loc[radish,'処理対象']=1

                    df.to_excel(file, index=True, header=True)
                    break
                elif key==27: # ESC で終了
                    df.to_excel(file, index=True, header=True)
                    break
        if key==27: # ESC で終了
                cv2.destroyAllWindows()
                break
        
    cv2.waitKey(1)
    
lx,ly,dx,dy =0,0,0,0
def onMouse(event, x, y, flag, params):
    global rdimg,head,bottom,angle,framed,angled,framing,angling,color,rect,lx,ly,dx,dy
    wname, img, gx,gy= params
    if framed == False:
        if event == cv2.EVENT_LBUTTONDOWN:
            framing = True # 矩形描画モードオン
            lx,ly = x,y
            print("矩形描画スタート")
        elif event == cv2.EVENT_MOUSEMOVE: 
            img = color.copy()
            h, w = rdimg.shape[0], rdimg.shape[1]
            cv2.line(img, (x,0),(x,h-1),(255, 0, 0))
            cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))       
            if framing == True :                      
                # img = color.copy()
                cv2.rectangle(img,(lx,ly),(x,y),(0,0,255),2)
                rect = (min(lx,x),min(ly,y),abs(lx-x),abs(ly-y))  
        elif event == cv2.EVENT_LBUTTONUP:
            if framing == True:
                framing = False
                framed = True
                print("フレーム一時確定")
                cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)
                angling = True
                head = rect[1]
                bottom = rect[1]+rect[3]

    elif angled == False:
        if event == cv2.EVENT_LBUTTONUP:
                if angling == True:
                    dx = gx - x
                    dy = gy - y
                    angled = True 
                    print("角度確定",dx,dy)
                    cv2.rectangle(img,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)
                    cv2.circle(img,(gx,gy), 3, (255,0,0), 3)
                    if x!=gx:
                        cv2.line(img, (x-500, y-int(500*(y-gy)/(x-gx))), (gx+500, gy+int(500*(y-gy)/(x-gx))), (255, 0, 0))
                    if dx == 0 :
                        angle = 0        
                    else:
                        if dx > 0:
                            angle = 180*np.arctan(dy/dx)/np.pi-90
                        else:
                            angle = 180*np.arctan(dy/dx)/np.pi+90
        elif event == cv2.EVENT_MOUSEMOVE:
                if angling == True : 
                    img = color.copy()
                    h, w = img.shape[0], img.shape[1]
                    if x!=gx:
                        cv2.line(img, (x-500, y-int(500*(y-gy)/(x-gx))), (gx+500, gy+int(500*(y-gy)/(x-gx))), (255, 0, 0))
                    cv2.line(img, (0, y), (w - 1, y), (255, 0, 0))
                    cv2.circle(img, (gx,gy), 3, (255,0,0), 3)
                    cv2.circle(img, (x,y), 3, (255,0,0), 3)   
        
    cv2.imshow(wname, img)
        
main(target=10)

NameError: name 'main' is not defined

In [ ]:
# df.to_excel('自動計測データ.xlsx', index=True, header=True)

In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
img = cv2.imread('test.png')
cv2.imshow('a',img)
key = cv2.waitKey(0)
print(key)
cv2.destroyWindow('a')
cv2.waitKey(1)

In [ ]:
abs(-np.array(23))